In [1]:
pip install --upgrade --force-reinstall numpy gensim pandas


  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached smart_open-7.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached wrapt-1.17.2-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata

# Consignas:
Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
Graficarlos.
Obtener conclusiones.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

In [16]:
import numpy as np

# Datos

In [3]:
def load_avengers_scripts(file_paths, movie_names):
    data = []
    for file_path, movie_name in zip(file_paths, movie_names):
        with open(file_path, 'r', encoding='latin-1') as f:
            lines = f.read().splitlines()
            for line in lines:
                if line.strip():
                    data.append({
                        "texto": line,
                        "pelicula": movie_name
                    })

    df = pd.DataFrame(data, columns=["texto", "pelicula"])
    return df

In [5]:
file_paths = [
        "/content/Avengers.Age.of.Ultron.txt",
        "/content/Avengers.Endgame.txt",
        "/content/Avengers.Infinity.War.txt",
        "/content/Avengers.txt"
    ]

movie_names = [
    "Avengers: Age of Ultron",
    "Avengers: Endgame",
    "Avengers: Infinity War",
    "The Avengers"
]

df = load_avengers_scripts(file_paths, movie_names)

print(df.head())
print(f"\nTotal de líneas combinadas: {len(df)}")

                                               texto                 pelicula
0                                (DISTANT EXPLOSION)  Avengers: Age of Ultron
1  STRUCKER ON PA: Report to your stations immedi...  Avengers: Age of Ultron
2                               This is not a drill.  Avengers: Age of Ultron
3                               We are under attack!  Avengers: Age of Ultron
4                   (SOLDIERS SHOUTING INDISTINCTLY)  Avengers: Age of Ultron

Total de líneas combinadas: 7793


# Preprocesamiento

In [6]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

In [7]:
sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

<ipython-input-7-23e21f4224e1>:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentence_tokens.append(text_to_word_sequence(row[0]))


In [8]:
sentence_tokens[:2]

[['distant', 'explosion'],
 ['strucker', 'on', 'pa', 'report', 'to', 'your', 'stations', 'immediately']]

# 2 - Crear los vectores (word2vec)

In [9]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [10]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)

In [11]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [12]:

# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 7793


In [13]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 930


# 3 - Entrenar embeddings

In [23]:
from gensim.models.callbacks import CallbackAny2Vec
import numpy as np

class EarlyStoppingCallback(CallbackAny2Vec):
    def __init__(self, patience=3, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = np.inf
        self.wait = 0
        self.epoch = 0
        self.loss_previous = 0.0
        self.loss_history = []

    def on_epoch_end(self, model):
        current_loss = model.get_latest_training_loss()
        epoch_loss = current_loss - self.loss_previous if self.epoch > 0 else current_loss
        self.loss_history.append(epoch_loss)

        print(f"Epoch {self.epoch + 1} - Loss: {epoch_loss:.4f}")

        if self.best_loss - epoch_loss > self.min_delta:
            self.best_loss = epoch_loss
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                print(f"Early stopping at epoch {self.epoch + 1}")
                raise StopIteration  # Gensim lo va a capturar y cortar el entrenamiento

        self.loss_previous = current_loss
        self.epoch += 1


In [24]:
early_stop = EarlyStoppingCallback(patience=5, min_delta=0.01)

In [25]:
try:
    w2v_model.train(
        sentence_tokens,
        total_examples=w2v_model.corpus_count,
        epochs=100,
        compute_loss=True,
        callbacks=[callback(), EarlyStoppingCallback(patience=5, min_delta=0.01)]
    )
except StopIteration:
    print("Entrenamiento detenido por Early Stopping.")


Loss after epoch 0: 129888.1015625
Epoch 1 - Loss: 129888.1016
Loss after epoch 1: 127880.1953125
Epoch 2 - Loss: 127880.1953
Loss after epoch 2: 128142.890625
Epoch 3 - Loss: 128142.8906
Loss after epoch 3: 127139.03125
Epoch 4 - Loss: 127139.0312
Loss after epoch 4: 125698.78125
Epoch 5 - Loss: 125698.7812
Loss after epoch 5: 124499.25
Epoch 6 - Loss: 124499.2500
Loss after epoch 6: 125712.4375
Epoch 7 - Loss: 125712.4375
Loss after epoch 7: 124422.375
Epoch 8 - Loss: 124422.3750
Loss after epoch 8: 119965.4375
Epoch 9 - Loss: 119965.4375
Loss after epoch 9: 120029.25
Epoch 10 - Loss: 120029.2500
Loss after epoch 10: 119626.375
Epoch 11 - Loss: 119626.3750
Loss after epoch 11: 119445.25
Epoch 12 - Loss: 119445.2500
Loss after epoch 12: 121587.125
Epoch 13 - Loss: 121587.1250
Loss after epoch 13: 119591.625
Epoch 14 - Loss: 119591.6250
Loss after epoch 14: 119542.25
Epoch 15 - Loss: 119542.2500
Loss after epoch 15: 118991.625
Epoch 16 - Loss: 118991.6250
Loss after epoch 16: 120052.87

# 4 - Ensayar

In [26]:
w2v_model.wv.most_similar(positive=["thor"], topn=10)

[("everyone's", 0.41773292422294617),
 ('package', 0.41563108563423157),
 ('potts', 0.41021886467933655),
 ('howard', 0.40618517994880676),
 ('jet', 0.39711329340934753),
 ('bigger', 0.38838642835617065),
 ('heading', 0.3873836100101471),
 ('dinner', 0.38202062249183655),
 ('copy', 0.37993738055229187),
 ('mr', 0.37983185052871704)]

➡️ Conclusión: el modelo aprendió que "thor" se comporta como un nombre de personaje importante, aunque no logra una agrupación semántica muy precisa (no aparecen dioses, héroes o enemigos directos como "loki", "odin", etc.).

In [27]:
w2v_model.wv.most_similar(positive=["hammer"], topn=10)

[('game', 0.5320076942443848),
 ('hero', 0.5127865672111511),
 ('deep', 0.49481895565986633),
 ('elevator', 0.49120378494262695),
 ('powering', 0.48760756850242615),
 ('chitauri', 0.4864158630371094),
 ('portal', 0.482796311378479),
 ('men', 0.48011118173599243),
 ('axe', 0.46707069873809814),
 ('woman', 0.4629170298576355)]

🔍 Interpretación: Estas palabras están relacionadas con batallas, armas y elementos de acción o ciencia ficción. "axe" es una herramienta similar a "hammer", "chitauri" son enemigos en la saga, y "portal" aparece en escenas clave de pelea.

➡️ Conclusión: el modelo entendió "hammer" como un objeto asociado a lucha o poder, y lo relaciona con conceptos de combate o escenas épicas.

In [28]:
w2v_model.wv.most_similar(positive=["thanos"], topn=10)

[('yours', 0.42224380373954773),
 ('destiny', 0.4020102918148041),
 ('born', 0.393685519695282),
 ('saying', 0.39201289415359497),
 ('starts', 0.39084574580192566),
 ('heist', 0.3893267810344696),
 ('lila', 0.3791799247264862),
 ('backup', 0.3782801926136017),
 ('turned', 0.3722943365573883),
 ('fall', 0.3720740079879761)]

🔍 Interpretación: A diferencia de "thor" o "hammer", acá vemos una carga más filosófica o narrativa: "destiny", "born", "saying", "fall", lo cual concuerda con el personaje Thanos, que suele hablar de propósito, destino, sacrificio.

➡️ Conclusión: el modelo captó bien el tono reflexivo y dramático del personaje. No lo asocia tanto con enemigos o peleas, sino con temas existenciales.

In [29]:
w2v_model.wv.most_similar(positive=["reality"], topn=10)

[('reading', 0.5089521408081055),
 ('metal', 0.4953494966030121),
 ('secret', 0.46797508001327515),
 ('push', 0.4611196517944336),
 ('signature', 0.45417845249176025),
 ('rhodes', 0.44240427017211914),
 ('failure', 0.4386778771877289),
 ('pal', 0.4375308156013489),
 ('heroes', 0.43654781579971313),
 ('damn', 0.43092602491378784)]

🔍 Interpretación: Aunque “reality” podría esperarse que esté asociada al contexto del reality stone, el modelo no parece haber captado ese significado directamente. En cambio, lo asocia con:

Acciones: "reading", "push", "failure"

Elementos confidenciales: "secret", "signature"

➡️ Conclusión: el modelo asocia reality a conceptos ocultos, documentales o situaciones críticas, probablemente reflejando su uso en escenas con tensión, decisiones, o información reveladora.

In [30]:
w2v_model.wv.most_similar(positive=["vision"], topn=10)

[('outside', 0.450346440076828),
 ('situation', 0.44929176568984985),
 ('boy', 0.43770068883895874),
 ('gem', 0.43396615982055664),
 ('language', 0.4299742579460144),
 ('hydra', 0.4078928232192993),
 ('backup', 0.40571439266204834),
 ('somebody', 0.40481793880462646),
 ('field', 0.3897557556629181),
 ('strucker', 0.38845503330230713)]

➡️ Conclusión: el modelo representa a vision más como personaje asociado a análisis, tecnología y contexto bélico/científico, reflejando bien su rol en las películas.

In [32]:
w2v_model.wv.most_similar(positive=["iron"], topn=10)

[('unsafe', 0.5384253263473511),
 ('beyond', 0.5145337581634521),
 ('next', 0.5113049149513245),
 ('boom', 0.503582239151001),
 ('grunts', 0.4839480519294739),
 ('soldiers', 0.482605904340744),
 ('swear', 0.4809715151786804),
 ('definitely', 0.4772692620754242),
 ('metal', 0.4690863788127899),
 ('side', 0.46679747104644775)]

➡️ Conclusión: el modelo representa iron como un término ligado a guerra, peligro y futurismo, probablemente por su uso frecuente en frases de Iron Man o descripciones tecnológicas.

In [33]:
w2v_model.wv.most_similar(positive=["captain"], topn=10)

[('america', 0.6454978585243225),
 ('7', 0.49956169724464417),
 ('artificial', 0.4503221809864044),
 ('form', 0.44700801372528076),
 ('threat', 0.4435817301273346),
 ('whoa', 0.4395093321800232),
 ('floor', 0.43047085404396057),
 ('drax', 0.40974196791648865),
 ('free', 0.40720638632774353),
 ('easy', 0.40625327825546265)]

➡️ Conclusión: el modelo entendió captain como una figura clave asociada a America, libertad y amenazas, captando su rol de líder y su origen artificial dentro de la historia.

In [41]:
w2v_model.wv.most_similar(positive=["war"], topn=10)

[('history', 0.4878173768520355),
 ('machine', 0.4546477794647217),
 ('jump', 0.43689024448394775),
 ('also', 0.4345536530017853),
 ('starts', 0.43043407797813416),
 ('path', 0.4241401255130768),
 ('asgardian', 0.4198858439922333),
 ('heist', 0.41982656717300415),
 ('body', 0.4192831814289093),
 ('reactor', 0.4172009825706482)]

➡️ Conclusión: el modelo entendió war más por su contexto en la historia que como guerra literal, relacionándolo con estrategia, tecnología y personajes como War Machine.

In [44]:
w2v_model.wv.most_similar(positive=["stones"], topn=10)

[('infinity', 0.5245112180709839),
 ('cradle', 0.5156025886535645),
 ('reading', 0.45838484168052673),
 ('weeks', 0.4581387937068939),
 ('went', 0.44630536437034607),
 ('humans', 0.4406774044036865),
 ('whole', 0.44021517038345337),
 ("'em", 0.43786755204200745),
 ('days', 0.43601787090301514),
 ('together', 0.4350634515285492)]

➡️ Conclusión: el modelo entendió stones como parte de las infinity stones, con referencias a tiempo, humanidad y escenas donde se habla de reunirlas, mostrando que captó bien su peso en la trama.

# 5 - Visualizar

In [34]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [35]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

Exception ignored on calling ctypes callback function: <function ThreadpoolController._find_libraries_with_dl_iterate_phdr.<locals>.match_library_callback at 0x7d0734a5a160>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1005, in match_library_callback
    self._make_controller_from_path(filepath)
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1187, in _make_controller_from_path
    lib_controller = controller_class(
                     ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 114, in __init__
    self.dynlib = ctypes.CDLL(filepath, mode=_RTLD_NOLOAD)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: dlopen() error


In [36]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [45]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

# Conclusión general del modelo

El modelo en general aprendió bastante bien las relaciones entre palabras dentro del universo de Avengers. No se quedó solo con significados literales, sino que captó conexiones narrativas, personajes, objetos clave y escenas recurrentes. Por ejemplo, entendió que *iron* está más asociado a guerra y tecnología que al metal en sí, o que *war* tiene más que ver con estrategia y personajes como War Machine que con conflicto directo.

También se ven asociaciones interesantes con conceptos como *stones*, donde aparecen palabras relacionadas al tiempo, humanidad y el hecho de reunirlas, lo que muestra que el modelo absorbió la estructura de la historia y no solo las frases. Incluso en casos como *captain*, se notan conexiones con libertad, amenazas y su origen artificial, lo que marca que el modelo entendió bien su rol dentro del grupo.

En resumen, el modelo captó no solo las palabras, sino el contexto y la dinámica del guion. No es perfecto, pero muestra un aprendizaje bastante sólido sobre los vínculos clave del universo narrativo.
